In [2]:
#####################  PIPELINE (Author: Jinsha George) ########################
# Step 1: Import libraries and Load Data
# Step 2: Data Cleaning - Column: author
# Step 3: Data Cleaning - Column: narrator
# Step 4: New Column - Column: minutes
# Step 5: Data Validation - Column: releasedate
# Step 6: New Column - Column: rating
# Step 7: New Column - Column: number_of_ratings
# Step 8: Data type correctiong - Column: price
# Step 9: Export the transformed data

################################################################################

In [4]:
# Step 1: Import libraries and Load Data
import pandas as pd
import re

def load_data(filename):
  project_df = pd.read_csv(filename)
  return project_df

In [5]:
# Step 2: Data Cleaning for names

def name_cleaning(name):
  name = name.replace('Writtenby:', '')
  name = name.replace('Narratedby:', '')
  name = re.sub('([A-Z])', r' \1', name)
  return name

In [6]:
# Step 2: Data Cleaning - Column: author

def data_clean_author(project_df):
  project_df.author = project_df['author'].apply(name_cleaning)
  return project_df

In [7]:
# Step 3: Data Cleaning - Column: narrator

def data_clean_narrator(project_df):
  project_df.narrator = project_df['narrator'].apply(name_cleaning)
  return project_df

In [8]:
# Step 4: New Column - Column: minutes
#extract total minutes from time

def extract_mins(time):
  mins = 0
  if 'hr' in time.lower():
    time_list = [int(s) for s in time.split() if s.isdigit()]
    if len(time_list) == 1:
      mins = time_list[0]*60
    elif len(time_list)==2:
      mins = (time_list[0]*60) + time_list[1]
    else:
      mins = 0
  else:
    if 'min' in time.lower():
      time_list = [int(s) for s in time.split() if s.isdigit()]
      if len(time_list) == 1:
        mins = time_list[0]
      else:
        mins = 0
  return mins


In [9]:
# Step 4: New Column - Column: minutes
def create_minutes(project_df):
  project_df['minutes'] = project_df['time'].apply(extract_mins)
  return project_df

In [10]:
# Step 5: Data Validation - Column: releasedate
#Date Validation, format: dd-mm-yy
#Note: Year is not validated because of ambiguity

def check_date(date):
  date_list = [int(s) for s in date.split('-') if s.isdigit()]
  if len(date_list) == 3:
    if date_list[1] < 13:
        if date_list[1] in [1,3,5,7,8,10,12] and date_list[0] < 32:
          return True
        elif date_list[1] == 2:
          if date_list[2]%4==0 and date_list[0] < 30:
            return True
          elif date_list[0] < 29:
            return True
        elif date_list[1] in [4,6,9,11] and date_list[0] < 31:
          return True
  return False

In [11]:
# Step 5: Data Validation - Column: releasedate

def date_valid(project_df):
  project_df['date_valid'] = project_df['releasedate'].apply(check_date)

  date_invalid = project_df['date_valid'] == False
  project_df[date_invalid] #All dates are valid

  project_df = project_df.drop (['date_valid'], axis =1)

  project_df["releasedate"] = pd.to_datetime(project_df["releasedate"], format='%d-%m-%y')

  return project_df

In [12]:
#Step 6: New Column - Column: rating

def clean_rating_float(project_df):
  project_df[['rating','number_of_ratings']] = project_df['stars'].str.split('stars', 1, expand=True) #spliting to two columns, secon column used in next step
  project_df.rating = project_df['rating'].str.extract('([-+]?\d*\.?\d+)').astype('float').fillna(0)
  return project_df

In [13]:
#Step 7: New Column - Column: number_of_rating

def clean_number_of_rating_int(project_df):
  project_df.number_of_ratings = project_df['number_of_ratings'].str.extract('(\d+)').astype('float')
  project_df.number_of_ratings = project_df['number_of_ratings'].fillna(0).astype('int')
  return project_df

In [14]:
# Step 8: Data type correctiong - Column: price

def clean_price_float(project_df):
  project_df.price = project_df['price'].str.replace(',', '').str.extract(r'(\d+.\d+)').astype('float')
  project_df.price = project_df['price'].fillna(0)
  return project_df

In [18]:
# Step 9: Export the transformed data

def export_data(project_df,filename):
  filename = filename.replace('.csv', '_transformed.csv')
  project_df.to_csv(filename, index = False)

In [19]:
# Pipeline calling every step

def pipeline():
  filename = 'project_dataset.csv'
  project_df = load_data(filename)
  project_df = data_clean_author(project_df)
  project_df = data_clean_narrator(project_df)
  project_df = create_minutes(project_df)
  project_df = date_valid(project_df)
  project_df = clean_rating_float(project_df)
  project_df = clean_number_of_rating_int(project_df)
  project_df = clean_price_float(project_df)
  export_data(project_df,filename)

In [20]:
pipeline()

<ipython-input-12-01ca5d54911d>:4: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  project_df[['rating','number_of_ratings']] = project_df['stars'].str.split('stars', 1, expand=True) #spliting to two columns, secon column used in next step
